Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.717376
Minibatch accuracy: 6.2%
Validation accuracy: 8.9%
Minibatch loss at step 50: 1.953475
Minibatch accuracy: 31.2%
Validation accuracy: 45.9%
Minibatch loss at step 100: 1.492779
Minibatch accuracy: 37.5%
Validation accuracy: 61.8%
Minibatch loss at step 150: 0.700998
Minibatch accuracy: 81.2%
Validation accuracy: 74.5%
Minibatch loss at step 200: 0.681988
Minibatch accuracy: 68.8%
Validation accuracy: 76.0%
Minibatch loss at step 250: 1.377413
Minibatch accuracy: 68.8%
Validation accuracy: 77.3%
Minibatch loss at step 300: 0.545901
Minibatch accuracy: 81.2%
Validation accuracy: 79.3%
Minibatch loss at step 350: 0.188070
Minibatch accuracy: 87.5%
Validation accuracy: 78.4%
Minibatch loss at step 400: 0.328839
Minibatch accuracy: 93.8%
Validation accuracy: 79.3%
Minibatch loss at step 450: 0.444017
Minibatch accuracy: 81.2%
Validation accuracy: 81.1%
Minibatch loss at step 500: 0.583691
Minibatch accuracy: 81.2%
Validation accuracy: 80.8%
Min

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [7]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense, BatchNormalization, Activation

In [8]:
batch_size = 128


pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [9]:
model_1 = Sequential([
    Conv2D(filters= 32,kernel_size=(3,3), strides=1, padding='valid', activation='relu', input_shape=(28,28,1) ),
    MaxPool2D(pool_size=(2,2)),
    
    Dropout(rate=0.25),
    
    Flatten(),
    Dense(1024, activation='relu'),
    Dropout(rate=0.25),
    Dense(10, activation='softmax'),
    
])


model_1.compile( optimizer='adam'
              , loss='sparse_categorical_crossentropy'
              , metrics=['accuracy']
             )


model_2 = Sequential([
    Conv2D(filters= 32,kernel_size=(3,3), strides=1, padding='valid', activation='relu', input_shape=(28,28,1) ),
    MaxPool2D(pool_size=(2,2)),
    
    Dropout(rate=0.25),
    
    Flatten(),
    Dense(1024, activation='relu'),
    Dropout(rate=0.25),
    Dense(10, activation='softmax'),
    
])


model_2.compile( optimizer='adam'
              , loss='sparse_categorical_crossentropy'
              , metrics=['accuracy']
             )


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [10]:
import numpy as np
train_dataset_shape = train_dataset.shape
train_dataset = train_dataset.reshape((train_dataset_shape[0], train_dataset_shape[1], train_dataset_shape[2], 1))
valid_shape = valid_dataset.shape
valid_dataset = valid_dataset.reshape((valid_shape[0], valid_shape[1],valid_shape[2],1))
test_shape = test_dataset.shape
test_dataset = test_dataset.reshape((test_shape[0], test_shape[1],test_shape[2],1))

In [11]:
def my_generator(batch_size):
    for batch_index in np.random.choice(range(train_dataset.shape[0])
                                        ,size=(train_dataset.shape[0]//batch_size, batch_size)
                                        ,replace=False
                                       ):
        yield train_dataset[batch_index], train_labels[batch_index]
    

In [12]:
model_1.fit(  x=train_dataset
          , y=train_labels
          , batch_size=batch_size
          , epochs=5
          , validation_data=(valid_dataset,valid_labels))

Train on 200000 samples, validate on 10000 samples
Epoch 1/5
200000/200000 [==============================] - 307s 2ms/sample - loss: 0.4302 - acc: 0.8725 - val_loss: 0.3377 - val_acc: 0.8990
Epoch 2/5
200000/200000 [==============================] - 280s 1ms/sample - loss: 0.3107 - acc: 0.9068 - val_loss: 0.2929 - val_acc: 0.9131
Epoch 3/5
200000/200000 [==============================] - 286s 1ms/sample - loss: 0.2640 - acc: 0.9197 - val_loss: 0.2766 - val_acc: 0.9166
Epoch 4/5
200000/200000 [==============================] - 301s 2ms/sample - loss: 0.2250 - acc: 0.9302 - val_loss: 0.2659 - val_acc: 0.9217
Epoch 5/5
200000/200000 [==============================] - 265s 1ms/sample - loss: 0.1901 - acc: 0.9412 - val_loss: 0.2667 - val_acc: 0.9237


In [13]:
model_1.evaluate(test_dataset,test_labels)

10000/10000 [==============================] - 5s 477us/sample - loss: 0.1099 - acc: 0.9701s - loss: 0.


[0.10986645233016461, 0.9701]

In [14]:
model_2.fit_generator(my_generator(batch_size)
                    , train_dataset.shape[0]//batch_size//5
                    , epochs=5
                , validation_data=(valid_dataset,valid_labels))

Epoch 1/5
312/312 [==============================] - 79s 252ms/step - loss: 0.5860 - acc: 0.8288 - val_loss: 0.4357 - val_acc: 0.8712
Epoch 2/5
312/312 [==============================] - 72s 232ms/step - loss: 0.4227 - acc: 0.8746 - val_loss: 0.3812 - val_acc: 0.8843
Epoch 3/5
312/312 [==============================] - 65s 209ms/step - loss: 0.3891 - acc: 0.8847 - val_loss: 0.3715 - val_acc: 0.8864
Epoch 4/5
312/312 [==============================] - 72s 230ms/step - loss: 0.3729 - acc: 0.8884 - val_loss: 0.3351 - val_acc: 0.8987
Epoch 5/5
312/312 [==============================] - 54s 174ms/step - loss: 0.3516 - acc: 0.8957 - val_loss: 0.3205 - val_acc: 0.9031


In [15]:
model_2.evaluate(test_dataset,test_labels)

10000/10000 [==============================] - 5s 495us/sample - loss: 0.1548 - acc: 0.9545


[0.1547955856844783, 0.9545]

In [16]:

model_3 = Sequential([
    Conv2D(filters= 32,kernel_size=(3,3), strides=1, padding='valid', activation='relu', input_shape=(28,28,1) ),
    MaxPool2D(pool_size=(2,2)),
    
    Dropout(rate=0.25),
    
    Flatten(),
    Dense(1024, activation='relu'),
    Dropout(rate=0.25),
    Dense(10, activation='softmax'),
    
])


model_3.compile( optimizer='adam'
              , loss='sparse_categorical_crossentropy'
              , metrics=['accuracy']
             )

In [17]:
for i in range(2):
    model_3.fit_generator(my_generator(batch_size)
                    , train_dataset.shape[0]//batch_size//5
                    , epochs=5
                , validation_data=(valid_dataset,valid_labels))

Epoch 1/5
312/312 [==============================] - 73s 235ms/step - loss: 0.6044 - acc: 0.8259 - val_loss: 0.4436 - val_acc: 0.8681
Epoch 2/5
312/312 [==============================] - 69s 222ms/step - loss: 0.4322 - acc: 0.8715 - val_loss: 0.3809 - val_acc: 0.8864
Epoch 3/5
312/312 [==============================] - 67s 214ms/step - loss: 0.3861 - acc: 0.8853 - val_loss: 0.3535 - val_acc: 0.8945
Epoch 4/5
312/312 [==============================] - 65s 210ms/step - loss: 0.3692 - acc: 0.8893 - val_loss: 0.3438 - val_acc: 0.8971
Epoch 5/5
312/312 [==============================] - 60s 193ms/step - loss: 0.3564 - acc: 0.8945 - val_loss: 0.3266 - val_acc: 0.9026
Epoch 1/5
312/312 [==============================] - 54s 174ms/step - loss: 0.3138 - acc: 0.9051 - val_loss: 0.3167 - val_acc: 0.9069
Epoch 2/5
312/312 [==============================] - 55s 175ms/step - loss: 0.3140 - acc: 0.9052 - val_loss: 0.3106 - val_acc: 0.9073
Epoch 3/5
312/312 [==============================] - 62s 200ms

In [18]:
model_3.evaluate(test_dataset,test_labels)

10000/10000 [==============================] - 3s 335us/sample - loss: 0.1321 - acc: 0.9627


[0.13205413690134882, 0.9627]

##### Let us give 3 more rounds to mode_3 which would logically equivalent to the way traying was done for model_1.
i.e. training entire training dataset 5 times through 5 epochs  == 5 times full batches; with one full set of batches through 5 epochs

In [19]:
for i in range(2):
    model_3.fit_generator(my_generator(batch_size)
                    , train_dataset.shape[0]//batch_size//5
                    , epochs=5
                , validation_data=(valid_dataset,valid_labels))

Epoch 1/5
312/312 [==============================] - 64s 205ms/step - loss: 0.2616 - acc: 0.9208 - val_loss: 0.2859 - val_acc: 0.9138
Epoch 2/5
312/312 [==============================] - 61s 197ms/step - loss: 0.2534 - acc: 0.9220 - val_loss: 0.2859 - val_acc: 0.9149
Epoch 3/5
312/312 [==============================] - 59s 189ms/step - loss: 0.2644 - acc: 0.9194 - val_loss: 0.2805 - val_acc: 0.9151
Epoch 4/5
312/312 [==============================] - 59s 190ms/step - loss: 0.2714 - acc: 0.9173 - val_loss: 0.2824 - val_acc: 0.9146
Epoch 5/5
312/312 [==============================] - 58s 187ms/step - loss: 0.2596 - acc: 0.9211 - val_loss: 0.2725 - val_acc: 0.9179
Epoch 1/5
312/312 [==============================] - 66s 211ms/step - loss: 0.2128 - acc: 0.9336 - val_loss: 0.2744 - val_acc: 0.9192
Epoch 2/5
312/312 [==============================] - 58s 187ms/step - loss: 0.2221 - acc: 0.9309 - val_loss: 0.2769 - val_acc: 0.9175
Epoch 3/5
312/312 [==============================] - 55s 175ms

In [20]:
model_3.evaluate(test_dataset,test_labels)

10000/10000 [==============================] - 3s 342us/sample - loss: 0.1078 - acc: 0.9692


[0.10775131025463343, 0.9692]

* It is mere conincidental that batches gave early orientation to the final result. We just saw the snapshot of the entire dataset training. even model1 would have same result at the same snapshot. 

* We should make sure that each batch has balanced labels. We know for sure we have balanced dataset; we should extend the same in each batches.

* If possible we should also try find the outlier and try a technique by name batch normalization.

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [21]:
from collections import Counter
Counter(train_labels)

Counter({4: 20000,
         9: 20000,
         6: 20000,
         2: 20000,
         7: 20000,
         3: 20000,
         5: 20000,
         0: 20000,
         1: 20000,
         8: 20000})

In [22]:
for td, tl in my_generator(1024):
    cntr_dict = dict(Counter(tl))
    
    print(np.array(list(cntr_dict.values())).min(),
          np.array(list(cntr_dict.values())).max(),
        np.array(list(cntr_dict.values())).mean(),
        np.array(list(cntr_dict.values())).std())

96 123 102.4 7.592101158440923
85 123 102.4 11.85073837362044
86 117 102.4 9.404254356406998
83 123 102.4 11.783038657324349
88 115 102.4 8.114185110040317
92 113 102.4 6.57571288910944
83 119 102.4 10.864621484432856
84 118 102.4 10.170545708072895
87 114 102.4 7.512655988397179
93 114 102.4 6.814690014960329
90 115 102.4 7.310266752998826
94 114 102.4 6.931089380465383
81 118 102.4 11.271202242884296
87 112 102.4 6.988562083862459
85 116 102.4 8.924124606929242
77 124 102.4 13.580868897091968
92 116 102.4 7.83836717690617
84 126 102.4 10.82774214691133
96 110 102.4 4.363484845854286
81 118 102.4 10.278132126023678
88 121 102.4 9.210863151735564
85 120 102.4 10.509043724335722
86 118 102.4 10.827742146911332
84 119 102.4 10.150862032359617
84 110 102.4 7.2
91 111 102.4 5.9194594347794975
83 118 102.4 10.44222198576529
89 118 102.4 9.86103442849684
86 113 102.4 8.42852300228219
90 122 102.4 10.041912168506554
86 124 102.4 12.109500402576483
88 117 102.4 8.321057625085889
84 114 102.4 9

There is almost 1-6% deviation from the mean of the number of samples for a label. Hence we need not have to investigate much in this aspect of balanced batches when we have 10K sample/batch size.

In [23]:

model_4 = Sequential([
    Conv2D(filters= 32,kernel_size=(3,3), strides=1, padding='valid', activation='relu', input_shape=(28,28,1) ),
    MaxPool2D(pool_size=(2,2)),
    
    Dropout(rate=0.25),
    
    Flatten(),
    Dense(1024, activation='relu'),
    Dropout(rate=0.25),
    Dense(10, activation='softmax'),
    
])


model_4.compile( optimizer='adam'
              , loss='sparse_categorical_crossentropy'
              , metrics=['accuracy']
             )

In [24]:
model_4.fit(  x=train_dataset
          , y=train_labels
          , batch_size=10240
          , epochs=5
          , validation_data=(valid_dataset,valid_labels))
model_4.evaluate(test_dataset,test_labels)

Train on 200000 samples, validate on 10000 samples
Epoch 1/5
200000/200000 [==============================] - 177s 884us/sample - loss: 0.9746 - acc: 0.7132 - val_loss: 0.6466 - val_acc: 0.8241
Epoch 2/5
200000/200000 [==============================] - 167s 837us/sample - loss: 0.6018 - acc: 0.8311 - val_loss: 0.5339 - val_acc: 0.8480
Epoch 3/5
200000/200000 [==============================] - 152s 759us/sample - loss: 0.5115 - acc: 0.8536 - val_loss: 0.4722 - val_acc: 0.8648
Epoch 4/5
200000/200000 [==============================] - 158s 791us/sample - loss: 0.4600 - acc: 0.8672 - val_loss: 0.4325 - val_acc: 0.8725
Epoch 5/5
10000/10000 [==============================] - 3s 318us/sample - loss: 0.2137 - acc: 0.9406


[0.21372416182756424, 0.9406]

##### Observations:
* Very High (200000) batchsize resulted in dead python kernel 
* High (100000) batchsize resulted in laptop getting hung
*  moderately high (10240) resulted in low accuracy; this may be the fact that when there are too many X then specific features of a label might be lost/generalised thus reduction in loss does not occur.
* I need to 2 more reading on 5120, 2560

In [25]:

model_5 = Sequential([
    Conv2D(filters= 32,kernel_size=(3,3), strides=1, padding='valid', activation='relu', input_shape=(28,28,1) ),
    MaxPool2D(pool_size=(2,2)),
    
    Dropout(rate=0.25),
    
    Flatten(),
    Dense(1024, activation='relu'),
    Dropout(rate=0.25),
    Dense(10, activation='softmax'),
    
])


model_5.compile( optimizer='adam'
              , loss='sparse_categorical_crossentropy'
              , metrics=['accuracy']
             )
model_5.fit(  x=train_dataset
          , y=train_labels
          , batch_size=5120
          , epochs=5
          , validation_data=(valid_dataset,valid_labels))
model_5.evaluate(test_dataset,test_labels)

Train on 200000 samples, validate on 10000 samples
Epoch 1/5
200000/200000 [==============================] - 147s 734us/sample - loss: 0.7983 - acc: 0.7736 - val_loss: 0.5592 - val_acc: 0.8440
Epoch 2/5
200000/200000 [==============================] - 157s 787us/sample - loss: 0.5057 - acc: 0.8556 - val_loss: 0.4513 - val_acc: 0.8687
Epoch 3/5
200000/200000 [==============================] - 169s 843us/sample - loss: 0.4290 - acc: 0.8748 - val_loss: 0.4011 - val_acc: 0.8795
Epoch 4/5
200000/200000 [==============================] - 163s 817us/sample - loss: 0.3890 - acc: 0.8863 - val_loss: 0.3667 - val_acc: 0.8923
Epoch 5/5
10000/10000 [==============================] - 4s 421us/sample - loss: 0.1765 - acc: 0.9513


[0.1764532840460539, 0.9513]

In [26]:

model_6 = Sequential([
    Conv2D(filters= 32,kernel_size=(3,3), strides=1, padding='valid', activation='relu', input_shape=(28,28,1) ),
    MaxPool2D(pool_size=(2,2)),
    
    Dropout(rate=0.25),
    
    Flatten(),
    Dense(1024, activation='relu'),
    Dropout(rate=0.25),
    Dense(10, activation='softmax'),
    
])


model_6.compile( optimizer='adam'
              , loss='sparse_categorical_crossentropy'
              , metrics=['accuracy']
             )
model_6.fit(  x=train_dataset
          , y=train_labels
          , batch_size=2048
          , epochs=5
          , validation_data=(valid_dataset,valid_labels))
model_6.evaluate(test_dataset,test_labels)

Train on 200000 samples, validate on 10000 samples
Epoch 1/5
200000/200000 [==============================] - 169s 847us/sample - loss: 0.6346 - acc: 0.8195 - val_loss: 0.4538 - val_acc: 0.8677
Epoch 2/5
200000/200000 [==============================] - 145s 727us/sample - loss: 0.4114 - acc: 0.8799 - val_loss: 0.3717 - val_acc: 0.8897
Epoch 3/5
200000/200000 [==============================] - 148s 738us/sample - loss: 0.3549 - acc: 0.8961 - val_loss: 0.3399 - val_acc: 0.8978
Epoch 4/5
200000/200000 [==============================] - 160s 799us/sample - loss: 0.3224 - acc: 0.9047 - val_loss: 0.3208 - val_acc: 0.9039
Epoch 5/5
10000/10000 [==============================] - 4s 411us/sample - loss: 0.1325 - acc: 0.9633s


[0.13245965872108936, 0.9633]

##### Let us now introduce Batch normalization

In [27]:

model_7 = Sequential([
    Conv2D(filters= 32,kernel_size=(3,3), strides=1, padding='valid', activation='relu', input_shape=(28,28,1) ),
    MaxPool2D(pool_size=(2,2)),
    
    Dropout(rate=0.25),
    
    Flatten(),
    Dense(1024, activation=None),
    BatchNormalization(),
    Activation('relu'),
    
    Dropout(rate=0.25),
    
    Dense(10, activation=None),
    #BatchNormalization(),
    Activation('softmax'),
    
])


model_7.compile( optimizer='adam'
              , loss='sparse_categorical_crossentropy'
              , metrics=['accuracy']
             )

model_7.fit(  x=train_dataset
          , y=train_labels
          , batch_size=1024
          , epochs=5
          , validation_data=(valid_dataset,valid_labels))
model_7.evaluate(test_dataset,test_labels)

Train on 200000 samples, validate on 10000 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/5
200000/200000 [==============================] - 152s 758us/sample - loss: 0.4721 - acc: 0.8658 - val_loss: 0.5484 - val_acc: 0.8994
Epoch 2/5
200000/200000 [==============================] - 142s 708us/sample - loss: 0.3175 - acc: 0.9061 - val_loss: 0.3116 - val_acc: 0.9120
Epoch 3/5
200000/200000 [==============================] - 142s 711us/sample - loss: 0.2710 - acc: 0.9199 - val_loss: 0.2864 - val_acc: 0.9155
Epoch 4/5
200000/200000 [==============================] - 144s 720us/sample - loss: 0.2352 - acc: 0.9298 - val_loss: 0.2777 - val_acc: 0.9183
Epoch 5/5
10000/10000 [==============================] - 4s 362us/sample - loss: 0.1141 - acc: 0.9675


[0.11411828586272896, 0.9675]

###### It indicates that we can have various archictecture which would result in better result. I am stopping my analysis here because anyways I know that there are overlaps in training, validataion and testing dataset. So I will address the data wrangling first and then try out various architecture. Otherwise, the test accuracy is not reliable.